In [ ]:
%pip install overturemaps lonboard geopandas shapely

In [ ]:
%pip install google.generativeai

In [ ]:
import overturemaps as om
from overturemaps import core
import overturemaps
import pandas
import geopandas as gpd
from shapely import wkb
from lonboard import Map, PolygonLayer, ScatterplotLayer
import ipywidgets as widgets
import numpy as np
from IPython.display import display


In [ ]:
def create_map(dataset):
    layer = ScatterplotLayer.from_geopandas(
        dataset,
        get_fill_color=[255, 0, 0],
        radius_min_pixels=5,
    )

    view_state = {
        "longitude": (bbox[0] + bbox[2]) / 2,
        "latitude": (bbox[1] + bbox[3]) / 2,
        "zoom": 8,
        "pitch": 45,
    }
    m = Map(layer, view_state=view_state)
    return m

In [ ]:
# specify bounding box
bbox = 9.0894, 45.4642, 9.1494, 45.5242

In [ ]:
place_dataset = core.geodataframe("place", bbox=bbox)
print(place_dataset.shape)

In [ ]:
# Read the categories.txt file
with open('Data/categories.txt', 'r') as file:
    categories_data = file.readlines()
categories_data = categories_data[1:]  # Skip the header line
# print(np.shape(categories_data))


# Split the data by semicolons and extract the part before the semi-colon
categories_list = [entry.split(';')[0] for entry in categories_data]
print(str(categories_list))

# Print the extracted parts in a readable format
# for category in categories_list:
#     print(category)

In [ ]:
#visualize the map
create_map(place_dataset)

In [ ]:
# Pulling out the places with no alt_categories
no_alt = {}
number = 0
for i in range(len(place_dataset.id)):
    # print(f"Place: {place_dataset.names[i]['primary']}")
    # number += 1
    if place_dataset.categories[i]['alternate'] is None:
        # Create a dictionary with the primary name as the key and the category as the value
        no_alt[place_dataset.names[i]['primary']] = place_dataset.categories[i]['primary']

# print(number)

In [ ]:
# Number of places with no alt_categories
print(len(no_alt.keys()))

# Print the places with no alt_categories
print(str(dict(list(no_alt.items()))))



In [ ]:
# Set up the API key
api_key = <Your Gemini API Key>

In [ ]:
# Initialize the Generative AI API
import google.generativeai as genai
import dspy
import os

# Configuring Generative AI and DSPY with API key
genai.configure(api_key=api_key)
gemini_api_key = os.getenv("api_key")  # grabbing from environment

# Instantiate Gemini models via DSPY for two-phase prompting
llm_prompt = dspy.LM("gemini-2.5-pro", api_key=gemini_api_key)
llm_exec = dspy.LM("gemini-2.0-flash", api_key=gemini_api_key)
dspy.configure(lm=llm_exec)
genai.configure(api_key=api_key)
context = "You are a Category Suggestion Agent. Your job is to read a POI record that I provide you with and propose an alternative category for it which is a one step deaper categorization (Outputing 'italian restaurant' instead of 'restaurant') using its name and category."+\
        f"These are all the categories in the schema:{str(categories_list)}." +\
        "I am going to provide you with all the POIs with no alternative category, I will provided the names and the primary categorys in this format: 'name': 'primary category'" +\
        f"These are the POIs: {str(dict(list(no_alt.items())))}" +\
        "Please provide me with a category that best sutes the alternative category for these POIs from the list of categories I provided you with."+\
        "The response should be in the format: 'name': 'primary category' & 'alternate category'."

In [ ]:
# -- A. Use 2.5-pro to generate the executor prompt based on the given context
prompt_response = llm_prompt.generate(prompt=context)
full_text = prompt_response.text
if "EXECUTE_PROMPT:" in full_text:
    executor_prompt = full_text.split("EXECUTE_PROMPT:")[1].strip().strip('"')
else:
    executor_prompt = full_text

# -- B. Run executor_prompt with 2.0-flash via DSPY
execution_response = dspy.generate(prompt=executor_prompt)
print(execution_response.text)
